# Atlas Processing - Initial Image Creation

- Documentation: https://nilearn.github.io/auto_examples/01_plotting/plot_surf_atlas.html
- Note: nilearn's 'surface.load_surface_data' function does not suppport .mgh / .mgz files; must use mri_convert or mris_convert (part of Freesurfer) to first convert to an acceptable format, e.g. .nii

In [14]:
import nilearn
from nilearn import surface
from nilearn import plotting

import os
import numpy as np
import matplotlib.pyplot as plt

# import pandas as pd
# import png  # for reloading / working with previously saved images

## Set up directories

In [2]:
# === DIRECTORIES === #

# input data directories
# overall format:
# -- hbn_dir/sub-{EID}/label_subdir/parc_filename

hbn_dir = '/scratch/users/samjohns/HBN/BIDS_curated/derivatives/freesurfer'
label_subdir = '/label'
surf_subdir = '/surf'

curv_filename = 'lh.curv'
infl_filename = 'lh.inflated'
pial_filename = 'lh.pial'
parc_filename = 'lh.aparc.a2009s.annot'

# output data directories
out_data_dir = '/scratch/groups/jyeatman/samjohns-projects/data'
image_out_subdir = 'parc-images'
image_out_dir = out_data_dir + '/' + image_out_subdir
image_out_dir
os.makedirs(image_out_dir, exist_ok=True)  # ensure image output directory exists
assert os.path.exists(image_out_dir)

# === LABELS === #

# important:
# select a subset of labels that are visible in  ventral view
# label = 43 was borderline and removed for convenience
labels_to_plot = [2, 19, 21, 23, 24, 25, 30, 37, 38, 50, 51, 57, 58, 59, 60, 61, 63, 65]

In [5]:
fname = 'subjects-to-atlas.txt'
subjects = [s for s in os.listdir(hbn_dir) if 'sub-' in s]
subjects.sort()
with open(fname, 'w') as f:
    for s in subjects:
        f.write(s + '\n')

In [7]:
subjects = subjects[:10]

## Make images

In [15]:
def make_subject_images(sub):
    parc_path = f'{hbn_dir}/{sub}{label_subdir}/{parc_filename}'
    curv_path = f'{hbn_dir}/{sub}{surf_subdir}/{curv_filename}'
    infl_path = f'{hbn_dir}/{sub}{surf_subdir}/{infl_filename}'
    
        # check files exist
    if not (os.path.exists(parc_path) 
            and os.path.exists(curv_path) 
            and os.path.exists(infl_path)
           ):
        return

    parc = surface.load_surf_data(parc_path)
    curv = surface.load_surf_data(curv_path)
    infl = surface.load_surf_mesh(infl_path)
    
    selected_parc = np.array([labels_to_plot.index(l) if l in labels_to_plot else -1 for l in parc])
    
    fig, ax = plt.subplots(figsize=(8, 8))
    plotting.plot_surf_roi(infl, selected_parc
                           ,view=(210, 90)
                           # ,bg_map=test_curv
                           # ,bg_on_data=True 
                           ,figure=fig
                           ,cmap='tab20'
                           ,output_file=f'{image_out_dir}/{sub}-parc.png'   
                           # ,threshold=25.0
                           # colorbar=True
                          )
    
    fig, ax = plt.subplots(figsize=(8, 8))
    plotting.plot_surf_roi(infl, selected_parc
                           ,view=(210, 90)
                           ,bg_map=curv
                           # ,bg_on_data=True 
                           ,figure=fig
                           ,cmap='tab20'
                           ,threshold=25.0
                           ,output_file=f'{image_out_dir}/{sub}-curv.png'
                           # colorbar=True
                          )

In [16]:
# main loop: don't execute this cell unless you want to go grab a coffee
for sub in subjects:
    make_subject_images(sub)